In [1]:
class Board:
    def __init__(self, mat, blank):
        self.mat = []
        for row in mat:
            new_row = []
            for val in row:
                new_row.append(val)
            self.mat.append(new_row)
        self.blank = blank[:]
        
    def show(self):
        for row in self.mat:
            for i in range(3):
                print("+---", end="")
            print("+")
            for val in row:
                if val == 0:
                    print("|   ", end="")
                else:
                    print(f"| {val} ", end="")
            print("|")
        for i in range(3):
            print("+---", end="")
        print("+")
    
    def move(self, cmd):
        r, c = self.blank
        
        if cmd == 'u':
            if r == 0:
                return False
            self.mat[r][c] = self.mat[r-1][c]
            self.mat[r-1][c] = ' '
            self.blank = (r-1, c)
        elif cmd == 'd':
            if r == 2:
                return False
            self.mat[r][c] = self.mat[r+1][c]
            self.mat[r+1][c] = ' '
            self.blank = (r+1, c)
        elif cmd == 'l':
            if c == 0:
                return False
            self.mat[r][c] = self.mat[r][c-1]
            self.mat[r][c-1] = ' '
            self.blank = (r, c-1)
        elif cmd == 'r':
            if c == 2:
                return False
            self.mat[r][c] = self.mat[r][c+1]
            self.mat[r][c+1] = ' '
            self.blank = (r, c+1)
        else:
            return False
        
        return True
    
    def copy(self):
        return Board(self.mat, self.blank)
    
    def heuristic(self):
        goal = [
            ['1', '2', '3'],
            ['4', '5', '6'],
            ['7', '8', ' ']
        ]
        
        count = 0
        for i in range(3):
            for j in range(3):
                if (i, j) == self.blank:
                    continue
                val = int(self.mat[i][j]) - 1
                col = val%3
                row = val//3

                count += abs(row-i) + abs(col-j)
        
        return count
    
    def __eq__(self, obj):
        for i in range(3):
            for j in range(3):
                if self.mat[i][j] != obj.mat[i][j]:
                    return False
        return True

In [2]:
class Node:
    def __init__(self, ele, pri):
        self.ele = ele
        self.pri = pri
        self.left = None
        self.right = None

class PQueue:
    def __init__(self):
        self.root = None
    
    def isEmpty(self):
        return self.root == None
    
    def enqueue(self, ele, pri):
        new = Node(ele, pri)
        
        if self.isEmpty():
            self.root = new
        else:
            prev = None
            curr = self.root
            
            while curr != None:
                prev = curr
                if pri < curr.pri:
                    curr = curr.left
                else:
                    curr = curr.right
            
            if pri < prev.pri:
                prev.left = new
            else:
                prev.right = new

    def dequeue(self):
        if self.isEmpty():
            return None, None
        
        prev = None
        curr = None
        next = self.root
        
        while next != None:
            prev = curr
            curr = next
            next = next.left
        
        if prev == None:
            if curr.right != None:
                self.root = curr.right
            else:
                self.root = None
        elif curr.right != None:
            prev.left = curr.right
        else:
            prev.left = None
        
        ele, pri = curr.ele, curr.pri

        return ele, pri

In [3]:
def solveboard(board):
    moves = ['u', 'd', 'l', 'r']
    normalize = ['d', 'u', 'r', 'l']
    
    visited = []
    
    q = PQueue()
    q.enqueue((board.copy(), [board.copy()]), board.heuristic())
    
    visited.append(board.copy())

    while not q.isEmpty():
        ele, currfn = q.dequeue()
        curr, path = ele
        
        currhn = curr.heuristic()
        
        if currhn == 0:
            return path
        
        currgn = len(path)            
        
        for i in range(4):
            if curr.move(moves[i]):
                if curr not in visited:
                    visited.append(curr.copy())
                    new_path = path[:]
                    new_path.append(curr.copy())
                    currfn = currgn + curr.heuristic()
                    q.enqueue((curr.copy(), new_path), currfn)
                curr.move(normalize[i])  

In [4]:
initial = [
            ['3', '6', ' '],
            ['2', '4', '7'],
            ['5', '8', '1']
        ]

blank = (0, 2)

In [5]:
b = Board(initial, blank)

In [6]:
path = solveboard(b)

In [7]:
print(f'The given 8 puzzle can be solved in {len(path)-1} moves in the following way: ')
for b in path:
    print()
    b.show()

The given 8 puzzle can be solved in 22 moves in the following way: 

+---+---+---+
| 3 | 6 |   |
+---+---+---+
| 2 | 4 | 7 |
+---+---+---+
| 5 | 8 | 1 |
+---+---+---+

+---+---+---+
| 3 |   | 6 |
+---+---+---+
| 2 | 4 | 7 |
+---+---+---+
| 5 | 8 | 1 |
+---+---+---+

+---+---+---+
|   | 3 | 6 |
+---+---+---+
| 2 | 4 | 7 |
+---+---+---+
| 5 | 8 | 1 |
+---+---+---+

+---+---+---+
| 2 | 3 | 6 |
+---+---+---+
|   | 4 | 7 |
+---+---+---+
| 5 | 8 | 1 |
+---+---+---+

+---+---+---+
| 2 | 3 | 6 |
+---+---+---+
| 4 |   | 7 |
+---+---+---+
| 5 | 8 | 1 |
+---+---+---+

+---+---+---+
| 2 | 3 | 6 |
+---+---+---+
| 4 | 8 | 7 |
+---+---+---+
| 5 |   | 1 |
+---+---+---+

+---+---+---+
| 2 | 3 | 6 |
+---+---+---+
| 4 | 8 | 7 |
+---+---+---+
| 5 | 1 |   |
+---+---+---+

+---+---+---+
| 2 | 3 | 6 |
+---+---+---+
| 4 | 8 |   |
+---+---+---+
| 5 | 1 | 7 |
+---+---+---+

+---+---+---+
| 2 | 3 | 6 |
+---+---+---+
| 4 |   | 8 |
+---+---+---+
| 5 | 1 | 7 |
+---+---+---+

+---+---+---+
| 2 | 3 | 6 |
+---+---+---